# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    total = 0
    
    
    for index, row in data.iterrows():
        #add weighted prices from each row
        total += row['Price in 1 Year'] * row['Probability']
        
    x_return = (total - P) / P
    # exp_return(data, 40) should output 0.07500000000000002
    return x_return

Write the function ```volatility```, which calculates the volatility of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    #Get the expected return of the stock
    x_return = exp_return(data, P)
        
    weight_sqr_dev = sum(data['Probability'] * ((data['Price in 1 Year'] / P - 1 - x_return) ** 2))
    
    volatile = weight_sqr_dev ** 0.5
    
    # volatility(data,40) should output 0.21301408404140795
    return volatile

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    x_return = exp_return(data, P)
    volatile = volatility(data, P)
    Z_score = 1.96
    
    #find std error
    std_err = Z_score * volatile /np.sqrt(len(data))
    
    #find upper and lower bounds
    l_bound = x_return - std_err
    u_bound = x_return + std_err

    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return [l_bound, u_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    
    x_return = rf + beta * (market_return - rf)
    
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return x_return

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # Write your code here
    df = pd.read_csv(path)
    
    #convert all RET to numbers
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    
    #drop NaN values
    df = df.dropna(subset = ['RET'])
    
    
    #filter for PERMNO with < 500 observations
    # Grouping by 'PERMNO' and creating a new column 'PERMNO_count' to represent counts
    df['PERMNO_count'] = df.groupby('PERMNO')['PERMNO'].transform('count')
    
    # Filter out rows where the count is less than 500
    df = df[df['PERMNO_count'] >= 500]
    
    # Drop the 'PERMNO_count' column as it was used for filtering
    df = df.drop(columns='PERMNO_count')
    
    # This should yield data with 403316 rows × 4 columns
    return df


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    data = clean_data(data)
    
    mean_returns = data.groupby('CUSIP')['RET']
    
    mean_returns = mean_returns.mean().reset_index()
    
    output = pd.DataFrame(data = {
        "CUSIP" : mean_returns['CUSIP'],
        "Average Return" : mean_returns['RET']
    })
    
    return output

In [9]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    data = clean_data(data)
    
    #group by CUSIP and calculate the std dev
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    
    #rename columns
    #volatilities = volatilities.set_axis(['Volatility'], axis=1)
    volatilities.rename(columns={'RET': 'Volatility'}, inplace=True)
    
    output = pd.DataFrame(data = {
        "CUSIP" : volatilities['CUSIP'],
        "Volatility" : volatilities['Volatility']
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    
    #Merge the dataframes
    df_merged = pd.merge(ret_data, vol_data, on='CUSIP')
    
    #find corr between average return and volatility
    corr = df_merged['Average Return'].corr(df_merged['Volatility'])
    
    return corr 